![logo](../LogoLine_horizon_C3S.png)

# Reproducing the single-system graphical products for additional variables

## Introduction

This Jupyter Notebook shows how the products shown in the C3S seasonal  __[graphical products](https://climate.copernicus.eu/charts/packages/c3s_seasonal/)__ can be calculated from data in the __[Climate Data Store](#configuration)__ (CDS), and plotted. The C3S seasonal graphical and data products are described in this __[documentation page](https://confluence.ecmwf.int/display/CKB/C3S+seasonal+forecast+product+descriptions)__. 

This example code can be used as the basis for creating graphical products which are not part of the C3S suite. In this example we look at monthly mean daily minimum temperature forecasts (for one system, ECMWF System 51), which is a variable available in the CDS dataset but not the graphical products. A 'tercile summary' is computed from tercile probabilities. An ensemble mean anomaly is also calculated and plotted with significance testing applied. This will be used in a further example to create multi-system combinations.

<div class="alert alert-block alert-warning">
<b>Note:</b> This Notebook produces equivalent outcomes to [](graphical_product_example_tmin.ipynb), but makes as much use of (Earthkit)[https://earthkit.readthedocs.io/en/latest/] as much as possible (instead of cf_grib to decode the data, xarray, and matplotlib). 
</div>

1. [Configuration](#configuration)
2. [CDS API requests](#cds-api-requests)
3. [Load forecast and hindcast data](#load-forecast-and-hindcast-data)
4. [Produce tercile summary](#produce-tercile-summary) 
5. [Plot tercile summary](#plot-tercile-summary) 
6. [Produce ensemble mean anomaly](#produce-ensemble-mean-anomaly)
7. [Plot ensemble mean anomaly](plot-ensemble-mean-anomaly)


## Configuration

Here we set which variable(s) will be downloaded, for which C3S seasonal system. We also set which forecast date we will create a forecast for, and which hindcast period to use (which will be used to calculate the terciles and anomalies). *Note that the URL and KEY need to be filled in with the details from your CDS account, and the cdsapi needs to be installed (NOTE that the CDS infrastructure including the API has recently been updated).*

<div class="alert alert-block alert-warning">
<b>Note:</b> To create multi-system combinations in the related Notebook this needs to be run for each of the system to be included. Simply edit the `prov = 'ecmf.s51' ` line, or add a loop which runs through multiple systems.
</div>

Import required modules and configure CDS API key and client.

In [ ]:
# !pip install 'cdsapi>=0.7.2'
# !pip install earthkit
# !pip list --local

In [2]:
# import required modules
import cdsapi
import os
import xarray as xr
import numpy as np
# import earthkit
import pandas as pd

# Date and time related libraries
from dateutil.relativedelta import relativedelta
from calendar import monthrange
import datetime

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# config to avoid issues saving to netcdf if needed
os.environ['HDF5_USE_FILE_LOCKING'] = 'FALSE'

# Need to leave this blank and the user needs to enter their url and key here
URL = 'https://cds.climate.copernicus.eu/api'
KEY = 'ebe2f300-f038-4dbb-94a1-9f21892d46da' # INSERT CDS KEY HERE
c = cdsapi.Client(url=URL, key=KEY)

2024-10-14 11:20:32,390 INFO [2024-09-28T00:00:00] **Welcome to the New Climate Data Store (CDS)!** This new system is in its early days of full operations and still undergoing enhancements and fine tuning. Some disruptions are to be expected. Your 
[feedback](https://jira.ecmwf.int/plugins/servlet/desk/portal/1/create/202) is key to improve the user experience on the new CDS for the benefit of everyone. Thank you.
2024-10-14 11:20:32,390 WARNING [2024-09-26T00:00:00] Should you have not yet migrated from the old CDS system to the new CDS, please check our [informative page](https://confluence.ecmwf.int/x/uINmFw) for guidance.
2024-10-14 11:20:32,391 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2024-10-14 11:20:32,392 INFO [2024-09-16T00:00:00] Remember that you need to have an ECMWF account to use the new CDS. **Your old CDS credentials will not work in new CDS!**
2024-10-14 11:20:32,393 WARNING [2024-06-16T

Define a library of provider details and variables of interest (only `tmin` is used here, but this will be compared to `tmean` and `tmax` in another Notebook).

In [3]:
# library of C3S systems and useful parameters
# max_hc and max_fc represent the number of members to be considered (the number index starts from 0, the most recent)
providers = {
    'ecmf.s51': {'cds_name': 'ecmwf', 'plot_name': 'ECMWF', 'plot_system': 'SEAS5', 'cds_system': '51',
                 'lagged': False},
    'lfpw.s8': {'cds_name': 'meteo_france', 'plot_name': 'Météo-France', 'plot_system': 'System 8', 'cds_system': '8',
                'lagged': False},
    'egrr.s602': {'cds_name': 'ukmo', 'plot_name': 'Met Office', 'plot_system': 'GloSea6', 'cds_system': '602',
                  'lagged': True, 'max_hc': 50, 'max_fc': 'none'},
    'edzw.s21': {'cds_name': 'dwd', 'plot_name': 'DWD', 'plot_system': 'GCFS2.1', 'cds_system': '21',
                 'lagged': False},
    'cmcc.s35': {'cds_name': 'cmcc', 'plot_name': 'CMCC', 'plot_system': 'SPS3.5', 'cds_system': '35',
                 'lagged': False},
    'kwbc.s2': {'cds_name': 'ncep', 'plot_name': 'NCEP', 'plot_system': 'CFSv2', 'cds_system': '2',
                'lagged': True, 'max_hc': 20, 'max_fc': 52},
    'rjtd.s3': {'cds_name': 'jma', 'plot_name': 'JMA', 'plot_system': 'CPS3', 'cds_system': '3',
                'lagged': True, 'max_hc': 'none', 'max_fc': 55},
    'cwao.s2': {'cds_name': 'eccc', 'plot_name': 'ECCC', 'plot_system': 'CanCM4i', 'cds_system': '2',
                'lagged': False},
    'cwao.s3': {'cds_name': 'eccc', 'plot_name': 'ECCC', 'plot_system': 'GEM5-NEMO', 'cds_system': '3',
                'lagged': False},
}

# some options for variables
vars = {
    'tmean': {'plot_name': 'daily mean T2m', 'cds_name': '2m_temperature'}, 
    'tmax' : {'plot_name': 'daily max T2m', 'cds_name': 'maximum_2m_temperature_in_the_last_24_hours'},
    'tmin' : {'plot_name': 'daily min T2m', 'cds_name': 'minimum_2m_temperature_in_the_last_24_hours'}
}

Set request details.

In [4]:
# select the provider from the dictionary above, 
# and the associated feilds needed for the CDS API request and loading the data
prov = 'ecmf.s51'  
centre = providers[prov]['cds_name']
version = providers[prov]['cds_system']
lagged = providers[prov]['lagged']

if lagged:
    st_dim_name = 'indexing_time'
else:
    st_dim_name = 'time'  # if not config.get('isLagged',False) else 'indexing_time'

# define some other parameters for the data request
fc_yr = '2024'
st_mon = '02'
var = 'tmin'
var_str = vars[var]['plot_name']   #for plotting
cds_var_name = vars[var]['cds_name']

lt_mons = [1, 2, 3, 4, 5, 6]  # cover all lead months when plotting month by month
hc_years = '1993_2016'  # string to print the hindcast period, and lable the file

# data path in cwd 
cwd = '~/code/jnbs/seasonal/products_production_jnb' # os.getcwd()
data_path = os.sep.join([ cwd, 'data', centre, version])
try:
   os.makedirs(data_path)
except FileExistsError:
   # directory already exists
   pass

## CDS API requests

Here we request the desired hindcast and forecast data in GRIB format using the CDS API, and save it within a folder 'data' in the current working directory, organised by originating centre and forecast system. For this example, the CDS API keywords used are:

**Format**: `Grib` <br>
**Variable**: `minimum_2m_temperature_in_the_last_24_hours` *set via 'cds_var_name'* <br>
**Originating centre**: `ECMWF` *set via 'centre'* <br>
**System**: `51` *this refers to SEAS5 system 51, set via 'version'* <br>
**Product type**: `Monthly mean` *all ensemble members will be retrieved* <br>
**Year**: `1993 to 2016` *for the hindcast* `2024` *for the forecast, set via 'hc_years' and 'fc_year'* <br>
**Month**: `02` *February, set via 'st_mon'* <br>
**Leadtime month**: `1 to 6` *all lead months available, February to July in this case*

In [5]:
import earthkit.data

dataset = 'seasonal-monthly-single-levels'
req = {
    'format': 'grib',
    'variable': cds_var_name,
    'originating_centre': centre,
    'system': version,
    'product_type': 'monthly_mean',
    'year': [
        '1993', '1994', '1995',
        '1996', '1997', '1998',
        '1999', '2000', '2001',
        '2002', '2003', '2004',
        '2005', '2006', '2007',
        '2008', '2009', '2010',
        '2011', '2012', '2013',
        '2014', '2015', '2016',
    ],
    'month': [st_mon],
    'leadtime_month': lt_mons
}

ds = earthkit.data.from_source('cds', dataset, req,  prompt=True)

2024-10-14 11:21:00,830 INFO [2024-09-28T00:00:00] **Welcome to the New Climate Data Store (CDS)!** This new system is in its early days of full operations and still undergoing enhancements and fine tuning. Some disruptions are to be expected. Your 
[feedback](https://jira.ecmwf.int/plugins/servlet/desk/portal/1/create/202) is key to improve the user experience on the new CDS for the benefit of everyone. Thank you.
2024-10-14 11:21:00,831 WARNING [2024-09-26T00:00:00] Should you have not yet migrated from the old CDS system to the new CDS, please check our [informative page](https://confluence.ecmwf.int/x/uINmFw) for guidance.
2024-10-14 11:21:00,832 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2024-10-14 11:21:00,833 INFO [2024-09-16T00:00:00] Remember that you need to have an ECMWF account to use the new CDS. **Your old CDS credentials will not work in new CDS!**
2024-10-14 11:21:00,833 WARNING [2024-06-16T

9b8cacf5727ff22e67d6205764b2421a.grib:   0%|          | 0.00/445M [00:00<?, ?B/s]

In [18]:
ds

GRIBReader(/etc/ecmwf/ssd/ssd1/jupyterhub/cxcg-jupyterhub/tmpdirs/cxcg.39899829/tmpkthp_uck/cds-retriever-d87e25f9192bbae6da4743d482a7801b29b3ec6f7681f696ccacb43f6f97ca06.grib)

In [6]:
ds.ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,mn2t24,surface,0,19930201,0,672,fcmean,0,regular_ll
1,ecmf,mn2t24,surface,0,19930201,0,672,fcmean,1,regular_ll
2,ecmf,mn2t24,surface,0,19930201,0,672,fcmean,2,regular_ll
3,ecmf,mn2t24,surface,0,19930201,0,672,fcmean,3,regular_ll
4,ecmf,mn2t24,surface,0,19930201,0,672,fcmean,4,regular_ll
...,...,...,...,...,...,...,...,...,...,...
3595,ecmf,mn2t24,surface,0,20160201,0,4368,fcmean,20,regular_ll
3596,ecmf,mn2t24,surface,0,20160201,0,4368,fcmean,21,regular_ll
3597,ecmf,mn2t24,surface,0,20160201,0,4368,fcmean,22,regular_ll
3598,ecmf,mn2t24,surface,0,20160201,0,4368,fcmean,23,regular_ll


In [7]:
ds.save(data_path + '/{}_mm_{}_{}_{}_{}_ek.grib'.format(var, centre, version, hc_years, st_mon))  # NOT WORKING!

In [20]:
data_path + '/{}_mm_{}_{}_{}_{}_ek.grib'.format(var, centre, version, hc_years, st_mon)

'~/code/jnbs/seasonal/products_production_jnb/data/ecmwf/51/tmin_mm_ecmwf_51_1993_2016_02_ek.grib'

In [9]:
xr_ds = ds.to_xarray(xarray_open_dataset_kwargs={"backend_kwargs": dict(time_dims=('forecastMonth', st_dim_name))})
xr_ds

<xarray.Dataset> Size: 933MB
Dimensions:        (number: 25, forecastMonth: 6, time: 24, surface: 1,
                    latitude: 180, longitude: 360)
Coordinates:
  * number         (number) int64 200B 0 1 2 3 4 5 6 7 ... 18 19 20 21 22 23 24
  * forecastMonth  (forecastMonth) int64 48B 1 2 3 4 5 6
  * time           (time) datetime64[ns] 192B 1993-02-01 ... 2016-02-01
  * surface        (surface) float64 8B 0.0
  * latitude       (latitude) float64 1kB 89.5 88.5 87.5 ... -87.5 -88.5 -89.5
  * longitude      (longitude) float64 3kB 0.5 1.5 2.5 3.5 ... 357.5 358.5 359.5
Data variables:
    mn2t24         (number, forecastMonth, time, surface, latitude, longitude) float32 933MB ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-10-14T11:27 GRIB to CDM+CF via cfgrib-0.9.1...

In [11]:
xr_ds.to_netcdf(data_path + '/{}_mm_{}_{}_{}_{}_ek.nc'.format(var, centre, version, fc_yr, st_mon))

In [12]:
dataset = 'seasonal-monthly-single-levels'
req = {
    'format': 'grib',
    'variable': cds_var_name,
    'originating_centre': centre,
    'system': version,
    'product_type': 'monthly_mean',
    'year': [fc_yr],
    'month': [st_mon],
    'leadtime_month': lt_mons
}

ds = earthkit.data.from_source('cds', dataset, req,  prompt=True)

2024-10-14 11:42:02,747 INFO [2024-09-28T00:00:00] **Welcome to the New Climate Data Store (CDS)!** This new system is in its early days of full operations and still undergoing enhancements and fine tuning. Some disruptions are to be expected. Your 
[feedback](https://jira.ecmwf.int/plugins/servlet/desk/portal/1/create/202) is key to improve the user experience on the new CDS for the benefit of everyone. Thank you.
2024-10-14 11:42:02,747 WARNING [2024-09-26T00:00:00] Should you have not yet migrated from the old CDS system to the new CDS, please check our [informative page](https://confluence.ecmwf.int/x/uINmFw) for guidance.
2024-10-14 11:42:02,748 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2024-10-14 11:42:02,749 INFO [2024-09-16T00:00:00] Remember that you need to have an ECMWF account to use the new CDS. **Your old CDS credentials will not work in new CDS!**
2024-10-14 11:42:02,750 WARNING [2024-06-16T

61874ffe2a09b1b6825de8438fe85f14.grib:   0%|          | 0.00/37.9M [00:00<?, ?B/s]

In [32]:
ds.ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,mn2t24,surface,0,20240201,0,696,fcmean,0,regular_ll
1,ecmf,mn2t24,surface,0,20240201,0,696,fcmean,1,regular_ll
2,ecmf,mn2t24,surface,0,20240201,0,696,fcmean,2,regular_ll
3,ecmf,mn2t24,surface,0,20240201,0,696,fcmean,3,regular_ll
4,ecmf,mn2t24,surface,0,20240201,0,696,fcmean,4,regular_ll
...,...,...,...,...,...,...,...,...,...,...
301,ecmf,mn2t24,surface,0,20240201,0,4368,fcmean,46,regular_ll
302,ecmf,mn2t24,surface,0,20240201,0,4368,fcmean,47,regular_ll
303,ecmf,mn2t24,surface,0,20240201,0,4368,fcmean,48,regular_ll
304,ecmf,mn2t24,surface,0,20240201,0,4368,fcmean,49,regular_ll


In [36]:
ds.save('~/code/jnbs/seasonal/products_production_jnb/test_ek.grib')  # NOT WORKING - no error! works in cwd without ~

In [19]:
xr_ds = ds.to_xarray(xarray_open_dataset_kwargs={"backend_kwargs": dict(time_dims=('forecastMonth', st_dim_name))})
xr_ds.to_netcdf(data_path + '/{}_mm_{}_{}_{}_{}_ek.nc'.format(var, centre, version, fc_yr, st_mon))

## Load forecast and hindcast data



First, prepare for the lagged start time coordinates, where careful treatment is required when loading the data into xarray (if needed). MOVED THIS EARLIER.

Now load the hindcast data as an xarray object using cfgrib: **Update to new CDS approach?**

In [24]:
# load hindcasts, sort time coords?
hc_path = data_path + '/' + '_'.join([var, 'mm', centre, version, hc_years, st_mon]) + '_ek.nc'
hcst = earthkit.data.from_source('file', hc_path)
hcst.ls()
# hcst = xr.open_dataarray(hc_path, engine='cfgrib',
#                          backend_kwargs=dict(time_dims=('forecastMonth', st_dim_name)))
# # rename indexing time or time to start date
# hcst = hcst.rename({st_dim_name: 'start_date', 'longitude': 'lon', 'latitude': 'lat'})
# # roll longitude
# hcst = hcst.assign_coords(lon=(((hcst.lon + 180) % 360) - 180)).sortby('lon')
# hcst

,variable,level,valid_datetime,units
0,mn2t24,None,1993-02-01T00:00:00,K
1,mn2t24,None,1994-02-01T00:00:00,K
2,mn2t24,None,1995-02-01T00:00:00,K
3,mn2t24,None,1996-02-01T00:00:00,K
4,mn2t24,None,1997-02-01T00:00:00,K
...,...,...,...,...
3595,mn2t24,None,2012-02-01T00:00:00,K
3596,mn2t24,None,2013-02-01T00:00:00,K
3597,mn2t24,None,2014-02-01T00:00:00,K
3598,mn2t24,None,2015-02-01T00:00:00,K


Now load the forecast data in the same manner:

In [27]:
fc_path = data_path + '/' + '_'.join([var, 'mm', centre, version, fc_yr, st_mon]) + '_ek.nc'
fcst = earthkit.data.from_source('file', fc_path)
fcst.ls()
# # rename indexing time or time to start date
# fcst = fcst.rename({st_dim_name: 'start_date', 'longitude': 'lon', 'latitude': 'lat'})
# # roll longitude
# fcst = fcst.assign_coords(lon=(((fcst.lon + 180) % 360) - 180)).sortby('lon')
# fcst

,variable,level,valid_datetime,units
0,mn2t24,None,2024-02-01T00:00:00,K
1,mn2t24,None,2024-02-01T00:00:00,K
2,mn2t24,None,2024-02-01T00:00:00,K
3,mn2t24,None,2024-02-01T00:00:00,K
4,mn2t24,None,2024-02-01T00:00:00,K
...,...,...,...,...
301,mn2t24,None,2024-02-01T00:00:00,K
302,mn2t24,None,2024-02-01T00:00:00,K
303,mn2t24,None,2024-02-01T00:00:00,K
304,mn2t24,None,2024-02-01T00:00:00,K


For the forecast array, we add a 'valid time' coordinate, which will be useful for labeling the plots later. 

In [31]:
fcst[0].metadata()

NetCDFMetadata({'GRIB_paramId': 52, 'GRIB_dataType': 'fcmean', 'GRIB_numberOfPoints': 64800, 'GRIB_typeOfLevel': 'surface', 'GRIB_stepUnits': 1, 'GRIB_stepType': 'instant', 'GRIB_gridType': 'regular_ll', 'GRIB_uvRelativeToGrid': 0, 'GRIB_NV': 0, 'GRIB_Nx': 360, 'GRIB_Ny': 180, 'GRIB_cfName': 'unknown', 'GRIB_cfVarName': 'mn2t24', 'GRIB_gridDefinitionDescription': 'Latitude/Longitude Grid', 'GRIB_iDirectionIncrementInDegrees': 1.0, 'GRIB_iScansNegatively': 0, 'GRIB_jDirectionIncrementInDegrees': 1.0, 'GRIB_jPointsAreConsecutive': 0, 'GRIB_jScansPositively': 0, 'GRIB_latitudeOfFirstGridPointInDegrees': 89.5, 'GRIB_latitudeOfLastGridPointInDegrees': -89.5, 'GRIB_longitudeOfFirstGridPointInDegrees': 0.5, 'GRIB_longitudeOfLastGridPointInDegrees': 359.5, 'GRIB_missingValue': 9999, 'GRIB_name': 'Minimum temperature at 2 metres in the last 24 hours', 'GRIB_shortName': 'mn2t24', 'GRIB_totalNumber': 0, 'GRIB_units': 'K', 'long_name': 'Minimum temperature at 2 metres in the last 24 hours', 'units

In [ ]:
valid_time = [pd.to_datetime(fcst.start_date.values) + relativedelta(months=fcmonth-1) for fcmonth in fcst.forecastMonth]
fcst = fcst.assign_coords(valid_time=('forecastMonth',valid_time))
fcst

### Select qualifying ensemble members

Here we select which ensemble members to use. For some lagged start systems a subset of the available ensemble members are used in the graphical products, as described in the __[documentation](https://confluence.ecmwf.int/display/CKB/Summary+of+available+data#:~:text=per%20forecast%20system-,Nominal%20Start%20Dates,-For%20systems%20with)__. 

In [ ]:
# for some lagged systems sub-select 'qualifying' ensemble members used in the C3S graphical products
if lagged:
    max_hc = providers[prov]['max_hc']
    max_fc = providers[prov]['max_fc']
    if isinstance(max_hc, int):
        hcst = hcst.sel(number=slice(None, max_hc-1))
    if isinstance(max_fc, int):
        fcst = fcst.sel(number=slice(None, max_fc-1))

## Produce tercile summary

### Calculate tercile summary from hindcasts

The first step of calculating probabilities for the tercile summary is to calculate the terciles from the hindcast period.

In [ ]:
quantiles = [1. / 3., 2. / 3.]
hcst_qbnds = hcst.quantile(quantiles, ['start_date', 'number'])
hcst_qbnds

### Compute forecast probabilities

Now we calculate the forecast probabilities by counting the number of ensemble members above and below the upper and lower quartiles.

In [ ]:
fc_ens = fcst.sizes['number']
print(" ####### FC ENS SIZE ####### ")
print(fc_ens)
# ens_size = len(fcst_coords['realization']['data'])
above = fcst.where(fcst > hcst_qbnds.sel(quantile=2. / 3.)).count('number')
below = fcst.where(fcst < hcst_qbnds.sel(quantile=1. / 3.)).count('number')

Now we calculate probabilites for each tercile category (above, below, and normal), and combine them to create a single array to use for plotting the tercile summary.

In [ ]:
P_above = 100. * (above / float(fc_ens))
P_below = 100. * (below / float(fc_ens))
P_normal = 100. - (P_above + P_below)

a = P_above.where(P_above > np.maximum(40. + 0 * P_above, P_below), 0.)
b = P_below.where(P_below > np.maximum(40. + 0 * P_below, P_above), 0.)
P_summary = a - b

P_summary

## Plot tercile summary

### Plot setup

Here we set up a directory for saving the plots, and set up some plot parameters.

In [ ]:
figpath = os.getcwd() + '/plots'
if not os.path.exists(figpath):
    os.makedirs(figpath)

In [ ]:
# levels to use when shading the plot, and corresponding colours
contour_levels = [-100., -70., -60., -50., -40., 40., 50., 60., 70., 100.]
contour_colours = ["navy", "blue", "deepskyblue", "cyan", "white", "yellow", "orange", "orangered", "tab:red"]

### Make a plot for each lead month

Note: the C3S graphical products are contour plots rather than the grid mesh plots below

In [ ]:
for ltm in lt_mons:
    lt_str = str(ltm).zfill(2)
    plot_data = P_summary.sel(forecastMonth=ltm)  # extract the specific forecast month
    
    valid_time = pd.to_datetime(plot_data.valid_time.values)
    vm_str = valid_time.strftime('%b')  # valid month string to label the plot
    title_txt1 = '{} system={}'.format(centre, version) + ', Probability most likely category ({})'.format(var_str)
    title_txt2 = 'start date = {}/{}, valid month: {} (leadtime_month = {})'.format(fc_yr, st_mon, vm_str, lt_str)

    fig = plt.figure(figsize=(16, 8))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.add_feature(cfeature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.add_feature(cfeature.COASTLINE, edgecolor='black', linewidth=2.)
    plot_data.plot(levels=contour_levels, colors=contour_colours,
                                               cbar_kwargs={'fraction': 0.033})
    plt.title(title_txt1 + '\n' + title_txt2)
    plt.tight_layout()
    figname = '_'.join(['tercile_summary', var_str, 'mm', centre, version, fc_yr, st_mon, 'lead', lt_str])
    plt.savefig(figpath + '/' + figname + '.png')


## Produce ensemble mean anomaly

Here we calculate the ensemble mean anomaly. To create a multi-system ensemble mean anomaly this needs to be done for each system, and save for use in another Jupyter Notebook (**retain this and link to the further example when ready?**).

In [ ]:
hcst_clim = hcst.mean(['start_date', 'number'])  # mean climatology for each lead time for the chosen start
fcst_anom = fcst - hcst_clim
fcst_anom_mean = fcst_anom.mean(['number'])
fcst_anom_mean

Save the ensemble mean anomaly for use in other Notebooks.

In [ ]:
save_name = '_'.join([var, centre, version, fc_yr, st_mon, 'ensmean_anom.nc'])
fcst_anom_mean.to_netcdf(os.sep.join([data_path, save_name]))

## Plot ensemble mean anomaly

### Plot setup

Note, the level values will need to be updated for variables other than Tmin/Tmax/Tmean.

In [ ]:
# specific values for Tmax, need dictionary for this
contour_levels = [-2., -1.5, -1., -0.5, -0.2, 0.2, 0.5, 1.0, 1.5, 2.0]
contour_colours = ["navy", "blue", "deepskyblue", "cyan", "white", "yellow", "orange", "orangered", "tab:red"]
# clip the data to -2 and 2
fcst_anom_mean = fcst_anom_mean.clip(min=-1.99, max=1.99)

### Make a plot for each lead month

As for the tercile probability summary, we plot a chart for each forcast month. Here no significance testing is applied, but this will be added in the next step (**in this case, should we only plot one month here, for comparison/to show the difference?**).

In [ ]:
for ltm in lt_mons:
    lt_str = str(ltm).zfill(2)
    plot_data = fcst_anom_mean.sel(forecastMonth=ltm)  # extract the specific forecast month
    
    valid_time = pd.to_datetime(plot_data.valid_time.values)
    vm_str = valid_time.strftime('%b')  # valid month string to label the plot
    title_txt1 = '{} system={}'.format(centre, version) + ', Ensemble mean anomaly ({})'.format(var_str)
    title_txt2 = 'Start date = {}/{}, valid month: {} (leadtime_month {})'.format(fc_yr, st_mon, vm_str, lt_str)

    fig = plt.figure(figsize=(16, 8))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.add_feature(cfeature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.add_feature(cfeature.COASTLINE, edgecolor='black', linewidth=2.)
    plot_data.plot(levels=contour_levels, colors=contour_colours,
                                               cbar_kwargs={'fraction': 0.033, 'extend': 'neither'})
    plt.title(title_txt1 + '\n' + title_txt2)
    plt.tight_layout()
    figname = '_'.join(['ens_mean', var_str, 'mm', centre, version, fc_yr, st_mon, 'lead', lt_str])
    plt.savefig(figpath + '/' + figname + '.png')

### Add significance contours and masking 

The single-system ensemble mean plots in the C3S seasonal graphical products also include significance testing, following the approach taken for SEAS5 products (see https://www.ecmwf.int/sites/default/files/medialibrary/2017-10/System5_guide.pdf). Anomaly values below the 10% significance level are masked out, and significance countours are drawn for 1% and 10%. Note, due to the significance testing this cell takes longer to run than the previous one. Note that the colorbar requires customisation to represent the masking in a similar manner to the graphical products.

In [ ]:
# Function for non-paired rank-sum test
from scipy.stats import mannwhitneyu

# Significance thresholds
pval_thresh_low = 0.1
pval_thresh_high = 0.01

for ltm in lt_mons:
    
    data1 = hcst.sel(forecastMonth=ltm)
    data2 = fcst.sel(forecastMonth=ltm)
 
    # need to flatten number and start date to simply 'samples'
    # proceed in numpy for simplicity
    data1 = data1.data  # includes member and start date
    data2 = data2.data  # includes member

    # flatten sample dimensions
    data1 = data1.reshape(-1, *data1.shape[-2:])  # lat lon are the last dimensions
    print('Flattened forecasts: ', data1.shape)

    # non-paired approach, need to use Mann–Whitney U test
    pvals2 = mannwhitneyu(data1, data2)
    pvals2.pvalue
    
    masked_anom_mean = fcst_anom_mean.sel(forecastMonth=ltm).where(pvals2.pvalue <= 0.1)

    # specific values for Tmax, adjusted to add a dummy white section to label as 'no signal'
    contour_levels = [-2., -1.5, -1., -0.5, -1e-15, 1e-15, 0.5, 1.0, 1.5, 2.0]  # dummy values to plot central white portion in colorbar
    cbar_colours = ["navy", "blue", "deepskyblue", "cyan", "white", "yellow", "orange", "orangered", "tab:red"]
    cbar_labels = ["<-1.5", "-1.5..-1.0", "-1.0..-0.5", "-0.5..0", "no signal", "0..0.5", "0.5..1.0", "1.0..1.5", ">1.5"]
    
    lt_str = str(ltm).zfill(2)
    valid_time = pd.to_datetime(plot_data.valid_time.values)
    vm_str = valid_time.strftime('%b')  # valid month string to label the plot
    title_txt1 = '{} system={}'.format(centre, version) + ', Ensemble mean anomaly ({})'.format(var_str)
    title_txt2 = 'Start date = {}/{}, valid month: {} (leadtime_month {})'.format(fc_yr, st_mon, vm_str, lt_str)
    print(title_txt1)
    print(title_txt2)

    fig = plt.figure(figsize=(16, 8))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.add_feature(cfeature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.add_feature(cfeature.COASTLINE, edgecolor='black', linewidth=2.)
    # masked_anom_mean.plot(levels=contour_levels, colors=contour_colours,
    #                                            cbar_kwargs={'fraction': 0.033, 'extend': 'neither'})
    im = masked_anom_mean.plot(levels=contour_levels, colors=cbar_colours, add_colorbar=False)
    cbar = plt.colorbar(im, fraction=0.023, extend='neither', ticks=[-1.75, -1.25, -0.75, -0.25, 0, 0.25, 0.75, 1.25, 1.75])
    cbar.ax.set_yticklabels(cbar_labels)
    
    plt.contour(masked_anom_mean.lon.values, masked_anom_mean.lat.values, pvals2.pvalue, levels=[0.01], colors='green')
    plt.title(title_txt1 + '\n' + title_txt2)
    plt.tight_layout()
    figname = '_'.join(['ens_mean', var_str, 'mm', centre, version, fc_yr, st_mon, 'lead', lt_str])
    plt.savefig(figpath + '/' + figname + '_masked.png')




These plots are similar to the monthly C3S graphical products. Note that here we have plotted the first month of the forecast (February, or leadtime_month = 1), while in the graphical products we show the monthly forecasts starting with the month following the release month (which would be March in this case). The three-month aggregations in the graphical products could be created in a similar manner, by combining the corresponding monthly means.